1. Імпорт всіх необхідних бібліотек:

In [30]:
import requests
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime
import os
import glob

2. Скачування датасету з сайту:


In [ ]:
def download():
    now = datetime.now()
    date_for_name = now.strftime("%d-%m-%Y-%H")
    cur_year, cur_week = now.isocalendar()[0], now.isocalendar()[1]

    pattern = "data/*_vhi_id_*.csv"  
    check_files = glob.glob(pattern)
    if check_files:
        for file in check_files:
            baza = os.path.basename(file)
            date_part = baza.split("_vhi_id_")[0]
            file_date = datetime.strptime(date_part, "%d-%m-%Y-%H")
            file_year, file_week = file_date.isocalendar()[0], file_date.isocalendar()[1]
            if (file_year, file_week) < (cur_year, cur_week):
                print("You have old data")
                all_files = glob.glob("data/*") 
                for old_file in all_files:
                    os.remove(old_file)
                print("all files was deleted")
                for i in range(1,28):
                    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2={cur_year}&type=Mean"
                    vhi_url = urllib.request.urlopen(url)  
                    out = open(f"data/{date_for_name}_vhi_id_{i}.csv", 'wb')
                    out.write(vhi_url.read())
                    out.close()
                    print(f"New VHI data for index {i} is downloaded...")
                break
    elif not check_files:
        print("You do not have data, wait for loading")
        for i in range(1, 28):
            url= f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2025&type=Mean"
            vhi_url = urllib.request.urlopen(url)  
            out = open(f"data/{date_for_name}_vhi_id_{i}.csv", 'wb')
            out.write(vhi_url.read())
            out.close()
            print(f"VHI data for index {i} is downloaded...")
            
download()
    

You have old data
all files was deleted
New VHI data for index 1 is downloaded...
New VHI data for index 2 is downloaded...
New VHI data for index 3 is downloaded...
New VHI data for index 4 is downloaded...
New VHI data for index 5 is downloaded...
New VHI data for index 6 is downloaded...
New VHI data for index 7 is downloaded...
New VHI data for index 8 is downloaded...
New VHI data for index 9 is downloaded...
New VHI data for index 10 is downloaded...
New VHI data for index 11 is downloaded...
New VHI data for index 12 is downloaded...
New VHI data for index 13 is downloaded...
New VHI data for index 14 is downloaded...
New VHI data for index 15 is downloaded...
New VHI data for index 16 is downloaded...
New VHI data for index 17 is downloaded...
New VHI data for index 18 is downloaded...
New VHI data for index 19 is downloaded...
New VHI data for index 20 is downloaded...
New VHI data for index 21 is downloaded...
New VHI data for index 22 is downloaded...
New VHI data for index 

3. Зчитування даних в датафрейм:

In [124]:
def dataframe():
    pattern = "data/*_vhi_id_*.csv"
    check_files2 = glob.glob(pattern)
    df_list = []
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    col = [0,1,2,3,4,5,6]
    for file in check_files2:
        area_id = int(file.split("_vhi_id_")[1].split(".csv")[0])
        df0 = pd.read_csv(file, header = 2, names = headers, usecols = col, skipfooter=3, engine='python')
        df0['Area'] = area_id
        df_list.append(df0)

    df = pd.concat(df_list, ignore_index = True)
    df["Area"] = df["Area"].replace({
        24:1, 25:2, 5:3, 6:4, 27:5, 23:6, 26:7, 7:8, 11:9, 13:10,
        14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17, 22:18, 8:19, 9:20, 
        10:21, 1:22, 3:23, 2:24, 4:25
    })
    df = df.drop(df.loc[df['VHI'] == -1].index)
    return df






4. VHI для певної області за вказаний рік


In [128]:
def task_1(df):
    try:
        index = int(input("Area index: "))
        year = int(input("Which year: "))
    except ValueError:
        print("Enter integer")
    else:
        if index <= 0 or year <= 0:
            print("Only positive and not zero") 

    df['Year'] = df['Year'].astype(int)
    f = df[(df["Area"] == index) & (df["Year"] == year)]["VHI"]
    print(f)
    



5. Екстремуми, середнє та медіана VHI для певної області та року



In [129]:
def task_2(df):
    try:
        index = int(input("Area index: "))
        year = int(input("Which year: "))
    except ValueError:
        print("Enter integer")
    else:
        if index <= 0 or year <= 0:
            print("Only positive and not zero")

    df['Year'] = df['Year'].astype(int)        
    min_v = df[(df["Year"] == year) & (df["Area"] == index) & (df["VHI"] != -1)]['VHI'].min()
    max_v = df[(df["Year"] == year) & (df["Area"] == index)]['VHI'].max()
    avg_v = df[(df["Year"] == year) & (df["Area"] == index)]['VHI'].mean()
    median = df[(df["Year"] == year) & (df["Area"] == index)]['VHI'].median()
    print(f"min: {min_v}, max: {max_v}, avg: {avg_v}, median: {median}")


6.


In [ ]:
def task_3(df):
    try:
        index = int(input("Area index: "))
        from_year = int(input("From year: "))
        to_year = int(input("To year: "))

    except ValueError:
        print("Enter integer")
    else:
        if index <= 0 or from_year <= 0 or to_year <= 0:
            print("Only positive and not zero")

    a = df.loc[(df["Year"] >= from_year) & (df["Year"] <= to_year) & (df["Area"] == index), "VHI"]
    print(a)

7.Для всього набору даних виявити роки, протягом яких екстремальні  посухи торкнулися більше вказаного відсотка областей по Україні (20%  областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI; 


Тобто є відсоток, його число множимо на кількість областей щоб визначити вибірку областей, тепер треба знайти всіііііі роки в яких VHI<=15  для >= 7 областей 

In [132]:
def task_4(df):
    try:
        procent = int(input("procent: "))

    except ValueError:
            print("Enter integer")
    else:
        if procent < 0 or procent > 100:
            print("Only positive and not zero")

    num = procent / 100
    sample = round(num * 27)
    print(sample)
    all = df[(df["VHI"] <= 15) & (df["VHI"] != -1)]
    count_uni_area = all.groupby("Year")["Area"].nunique() 
    fin = count_uni_area[count_uni_area >= sample].index
    print(fin)
    result = all[all["Year"].isin(fin)]
    print(result[["Year", "Area", "VHI"]])



In [140]:
df = dataframe()
#task_1(df)

In [139]:
#task_2(df)

In [138]:
#task_3(df)

In [137]:
#task_4(df)